#IBM Capstone Week 3 

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe. 
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
Submit a link to your Notebook on your Github repository. 

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.
Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.


In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup #scrapping the website
import requests

print('Libraries imported.')

In [ ]:
#data import
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_text = requests.get(url).text 

url_text_html = BeautifulSoup(url_text, "html.parser")

neigh_table = url_text_html.find("table", attrs={"class":"wikitable"})
neigh_table_info = neigh_table.tbody.find_all("tr")

neigh_info = []
for row in neigh_table_info:
    data = row.text.split("\n")[1:-1]
    neigh_info.append(data)

neigh_info[0:5]

In [ ]:
#dataframe columns
neigh_df = pd.DataFrame(neigh_info[1:], columns=neigh_info[0])
neigh_df.head()
neigh_df.shape

In [ ]:
#remove unassigned rows
Not_assign = neigh_df[neigh_df["Borough"]=="Not assigned"].index
neigh_df.drop(Not_assign, inplace=True)
neigh_df.shape

In [ ]:
neigh_df.loc[neigh_df["Neighborhood"]=="Not assigned", "Neighborhood"] = neigh_df["Borough"]
neigh_df.head()
neigh_df.shape

#Part 2
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. 

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [ ]:
!wget -q -O "geo_data.csv" http://cocl.us/Geospatial_data
geo_data = pd.read_csv("geo_data.csv")
geo_data.head()

In [ ]:
neigh_lat_long = pd.merge(geo_data,neigh_df,on="Postal Code")
neigh_lat_long = neigh_lat_long[["Postal Code","Borough","Neighborhood","Latitude","Longitude"]]
neigh_lat_long.head()

#Part 3
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
Just make sure:

1)to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2)to generate maps to visualize your neighborhoods and how they cluster together.

In [ ]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent = "to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of Toronto are {}, {}".format(latitude, longitude))

In [ ]:
#map of Toronto w lat and long
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers
for lat, lng, borough, neighborhood in zip(neigh_lat_long["Latitude"], neigh_lat_long["Longitude"], neigh_lat_long["Borough"], neigh_lat_long["Neighborhood"]):
    label = '{},{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [ ]:
#foursquare

CLIENT_ID = 'JXVDPJQP2QBNAN1CFTLQ0T332JLEN4GCO3DGRDN0PDATC3PX' # your Foursquare ID
CLIENT_SECRET = 'ORFBC2GRRZPU1TUH55CIOKJCECEBZNZP1S0A20TBQ4ZY5C3V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
#neighborhoods in Toronto 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        #create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                VERSION,
                lat,
                lng,
                radius,
                LIMIT)

        #make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]

        #return only relevant information for each nerby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v["venue"]["name"],
            v["venue"]["location"]["lat"],
            v["venue"]["location"]["lng"],
            v["venue"]["categories"][0]["name"]) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#new dataframe for each
toronto_venues= getNearbyVenues(names=neigh_lat_long["Neighborhood"],
                                latitudes=neigh_lat_long["Latitude"],
                                longitudes=neigh_lat_long["Longitude"])

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.rename(columns={"Neighbourhood":"Neighborhood","Neighbourhood Latitude":"Neighborhood Latitude","Neighbourhood Longitude":"Neighborhood Longitude"}, inplace=True)
toronto_venues.head()
toronto_venues.groupby("Neighborhood").count()

In [ ]:
#analyzing individually 
toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix="", prefix_sep="")
toronto_onehot["Neighborhood"]=toronto_venues["Neighborhood"]
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()
toronto_grouped.shape

In [ ]:
#print each with top 5 most common values
top_venues = 5

for hood in toronto_grouped["Neighborhood"]:
    print("-----"+hood+"-----")
    temp = toronto_grouped[toronto_grouped["Neighborhood"] == hood].T.reset_index()
    temp.columns = ["venue","freq"]
    temp = temp.iloc[1:]
    temp["freq"] = temp["freq"].astype(float)
    temp = temp.round({"freq":2})
    print(temp.sort_values("freq", ascending= False).reset_index(drop=True).head(top_venues))
    print("\n")

In [ ]:
#clustering neighborhoods
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]